# Run a pre-trained predictor

This notebook is for when you want to try my trained "best predictor" on new data in your possession. 

Be warned! I fully expect this to be quite bad at prediction... there are going to be batch differences between your data and ours, in terms of things that are controllable or potentially correctable (wetlab things, bioinformatics pipeline) and things that are not (e.g., your sample is composed of different genetic variants than mine, or subjects have very different epigentics due to environmental factors).  Maybe someday when I've collected an order of magnitude or two more samples the uncontrollable batch effects won't be as big a problem.  On that note, if you would like to contribute your samples to that endeavor please contact me: [jfleischer@salk.edu](mailto:jfleischer@salk.edu) :)

Requirements: FPKM/RPKM data collected in a single batch on a single platform, or maybe if you're lucky data you have batch corrected from multiple batches might work.  TPM might work, raw counts is almost certain to have huge issues. The data should ideally be top-expressing isoform counts per gene; maybe sum-of-isoform-expression-levels per gene works, but it's sure to be worse since that's not what I trained on.  To avoid problems where your batch has a different top isoform than my batch for a given gene, the data should be labelled with gene names rather than transcript/isoform names.  See my comments on loading data, in the next set of notebook cells. 

You'll have the best chance of this working if you use the same wetlab procedures and bioinformatics pipeline we used in [our paper.](https://link.springer.com/epdf/10.1186/s13059-018-1599-6).  

Yes, I know that's not realistic; your wetlab is going to be different and you can't ask them to rerun everything... probably you'll do better to use my other notebook, which allows you to train your own predictor on your unique data.  If you don't have enough data of your own to train a classifier (I'd say 3+ samples per decade samples, spread from 0 - 70 years would be a good minimum) perhaps you could consider [downloading our data](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE113957) and combining it with yours using some kind of batch correction software (like ComBat, SVA, etc).  I'd love to [hear from you](mailto:jfleischer@salk.edu) if you try this!!

This notebook requires versions of the following libraries that are at least as recent as:
* Jupyter notebook 5.0.0
* Python 2.7.9
* Numpy 1.15.2
* Pandas 0.22
* Sklearn 0.19
* Matplotlib 2.1.0
* Seaborn 0.7.1

Because this notebook will use a binary dump of the ensemble, using a later version of these libraries may result in putting the right bits in the wrong places... that is, if sklearn changes the underlying data structures in future versions the classifier might be broken even though it loads just fine.  I have tested the interchangability of sklearn 0.19 and 0.20, and  training on one and testing on the other produces identical results with my data, even though it throws a warning about the different sklearn versions during the loading process.  I make no guarantee that this will still work with future versions of sklearn.

We could solve this issue by serializing the ensemble and each of its components to a format like JSON of YAML instead of binary.  I know enough to know the problem isn't trivial; I hope to revisit it someday, or if you're a python expert, maybe you'd like to contribute some code?

In [4]:
%run age_predictors.py

There are two different ways we can try to match your sample expression data to our sample expression data: transcript-level or gene-level variables

In the paper we used **ONLY** the top expressing isoform for each gene.  The pre-trained classifier will always be trained with that kind of data... each variable will be called something like NM_XXXX or NR_XXXXX.  

We can supply a classifier that's looking for you to supply a transcript level variable as described above, and the notation will be RefSeq.  That is your data will have variables like NM_000600.  **OR** we can supply a classifier that's looking for you to supply a gene level label in your data, like IL6. 

Why would you choose one or the other?  Well if you have deep sequencing and can get all the isoforms reasonbly quantified, definitely go for the transcript level.  If you don't have enough depth to get all isoforms, then you should probably choose gene level, since there's a possibility for any given gene that you would have a different top isoform than our batch.   In that case you would be using fewer variables to make the prediction, which might results in a loss of predictive power.  Of course going to gene level representation may also be a loss of predictive power... there's always a chance that what changes with age is WHICH isoform gets expressed ;)  So in that case, maybe try it both ways?? \*shrug\*

In [ ]:
variable_type = # pick one of the following choices, and insert it here: 
# ['gene', 'transcript']

We can try to reduce batch effects by normalizing the data.  I've only coded up two very simple methods, and in my limited experience with different datasets neither one of them have been overwhelmingly effective.

Whichever transform you pick, DON'T change your data... you supply FPKM, my code will make the proper transform for you.  

The default is logFPKM, that's what we used in the paper.  The problem is that what if your sequencing platform is different than ours, and for some important gene your data ends up having expression values that are X% larger than ours on average... that's going to mess up the prediction.

So you can choose rank normalization to rank normalize per sample, essentially ordering each gene from highest to lowest expressing per person. See pandas.DataFrame.rank(axis=1) for additional details.

Or you can choose quantile normalization to normalize per gene, assigning each person to a quantile per gene in our original dataset; your dataset will then also be quantilized per gene, and your data will be transformed into the same FPKM values that would be present in our dataset for the same quantile.  See sklearn.preprocessing.QuantileTransformer() for additional details. 

In [ ]:
data_xform = # pick one of the following choices, and insert it here:
# ['logFPKM', 'rank', 'quantile']

Here's where you can define your own function that is suitable for loading your data in a format that works with this software

Requirements: Output data needs to be in a pandas Dataframe that contains a row for every sample and a column for every expression value.  The index of the dataframe should include a column 'UID' that uniquely identifies each sample, a column 'age' that expresses the subjects age in years, and a column 'meta' that holds metadata you might want to keep track of for a later analysis.   Column header must have a unique label for each column, where the label is the either the gene name or transcript name according to your choice of ```variable_type``` (see discussion above).  The corresponding expression data should ideally be isoform level, but maybe gene level would work.

This implementation assumes that the data will come from a file organized just like this, depending on your choice of ```variable_type```: 

| UID     | age | meta | gene1 | gene2 |   |
|---------|-----|------|-------|-------------|---|
| sample1 | 11  | healthy | ...         | ...         |   |
| sample2 | 13  | progeria | ...         | ..          |   |

OR

| UID     | age | meta | transcript1 | transcript2 |   |
|---------|-----|------|------|-------------|---|
| sample1 | 11  | healthy | ...         | ...         |   |
| sample2 | 13  | progeria | ...         | ..          |   |


If you have a file that is the opposite orientation, where rows are transcripts and columns are samples, then just set the argument transpose=True when you call load data.

This function will load comma seperated or xlsx files according to the ending of filename variable.

Feel free to modify this function to better fit the file format you prefer to use!


In [1]:
def load_data(filename, transpose=False):
    
    ending = filename.split('.')[-1]
    
    if ending == 'csv':
        data = pd.read_csv(filename,header=None,index_col=None)
    elif ending == 'xlsx':
        data = pd.read_xlsx(filename,header=None,index_col=None)
    else:
        raise TypeError('dont know what file type this is')
    
    if transpose:
        data = data.T
    
    # make sure the index columns are named correctly, 
    # otherwise use whatever the first row (header) contains for gene/transcript names
    cols = data.iloc[0,:]
    cols[0] = 'uid'
    cols[1] = 'age'
    cols[2] = 'meta'
    data.columns = cols
    
    # get the data, not the header now that we formed it
    data = data.iloc[1:,:]
    
    # make sure the age comes in as integer years... if you need to do floating point change this
    data.iloc[:,1] = data.iloc[:,1].astype(int)
    
    data = data.set_index(['uid','age','meta']).astype(float)
    
    return data



In [ ]:
# this won't work if you don't point this function at your data :)

data = load_data('path/to/folder/your_filename_here.csv')

In [5]:
# OK, let's load and test... 

fname='trained_ensembleLDA/'+variable_type+'_'+data_xform+'_small'
ensemble = load(fname)
# I keep having problem with git lfs and cant seem to push multi-GB files to github
# therefore I'm just going to have the classifier you choose retrain itself on the Hetzer data
# which given the amount of time it takes to download and then load a multi-GB file 
# this way shouldn't be too bad compared to the original plan
# on my computer (4.2Ghz i7 4 core, 24GB RAM) this takes about 6 minutes
print("Refitting on Hetzer data. Should take less than 10 min. Runtime warning about single sample is OK.")
%time ensemble.refit()

true = data.index.get_level_values('age').values
pred = ensemble.predict(data) 

sns.set_style('white')
fig1 = plt.figure(figsize=(3.5,3.5));
ax1 = fig1.add_subplot(111);
errstr = "MAE:{:3.1f} MED:{:3.1f} R2:{:3.2f}".format( score_MAE(true,pred), score_MED(true,pred), score_R2(true,pred)) 
sns.regplot(true,pred, ax=ax1)
ax1.plot([0,100],[0,100],'k:')
ax1.text(0.99, 0.03, errstr,
        horizontalalignment='right',
        verticalalignment='bottom',
        transform=ax1.transAxes)
plt.axis('equal')
ax1.set_xlabel('True age (years)'), ax1.set_ylabel('Predicted age (years)');
sns.despine()
plt.tight_layout()